<a href="https://colab.research.google.com/github/arshith26/arshith/blob/main/_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!pip install gradio numpy pillow matplotlib

# Import libraries
import gradio as gr
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from matplotlib import cm

# Add hidden text or logo to depth map
def embed_message(depth_map, text=None, logo=None):
    depth_map = depth_map.convert('L')  # Ensure grayscale
    canvas = ImageDraw.Draw(depth_map)

    if text:
        font = ImageFont.load_default()
        text_size = canvas.textsize(text, font=font)
        position = ((depth_map.width - text_size[0]) // 2, (depth_map.height - text_size[1]) // 2)
        canvas.text(position, text, fill=128, font=font)

    if logo:
        logo = logo.convert('L').resize((depth_map.width // 4, depth_map.height // 4))
        x_offset = (depth_map.width - logo.width) // 2
        y_offset = (depth_map.height - logo.height) // 2
        depth_map.paste(logo, (x_offset, y_offset), logo)

    return np.array(depth_map) / 255.0  # Normalize for further processing

# Generate a color stereogram
def create_stereogram(depth_map, pattern, depth_shift):
    h, w = depth_map.shape
    ph, pw, _ = pattern.shape
    stereogram = np.zeros((h, w, 3), dtype=np.uint8)

    for y in range(h):
        for x in range(w):
            shift = int(depth_map[y, x] * depth_shift)
            stereogram[y, x] = pattern[y % ph, (x - shift) % pw]

    return Image.fromarray(stereogram)

# Apply colormap for visualization
def apply_colormap(image, scheme):
    image_array = np.array(image) / 255.0
    cmap = cm.get_cmap(scheme)
    colorized = (cmap(image_array)[:, :, :3] * 255).astype(np.uint8)
    return Image.fromarray(colorized)

# Stereogram generation workflow
def generate_stereogram(
    depth_map,
    pattern=None,
    logo=None,
    text=None,
    depth_scale=30,
    pattern_size=50,
    color_scheme="grayscale",
    output_width=800
):
    output_height = int(output_width / 2.67)
    depth_map = depth_map.resize((output_width, output_height)).convert('L')
    depth_data = embed_message(depth_map, text, logo)

    if pattern:
        pattern = pattern.resize((pattern_size, pattern_size)).convert('RGB')
        pattern_array = np.array(pattern)
    else:
        pattern_array = (np.random.rand(pattern_size, pattern_size, 3) * 255).astype(np.uint8)

    stereogram = create_stereogram(depth_data, pattern_array, depth_scale)

    if color_scheme != "grayscale":
        depth_visual = apply_colormap(Image.fromarray((depth_data * 255).astype(np.uint8)), color_scheme)
    else:
        depth_visual = Image.fromarray((depth_data * 255).astype(np.uint8))

    return depth_visual, Image.fromarray(pattern_array), stereogram

# Gradio app setup
with gr.Blocks() as app:
    gr.Markdown("# Customizable Color Autostereogram Generator")

    with gr.Row():
        depth_input = gr.Image(label="Upload Depth Map", type="pil")
        pattern_input = gr.Image(label="Upload Pattern Image (Optional)", type="pil")
        logo_input = gr.Image(label="Upload Logo (Optional)", type="pil")
        text_input = gr.Textbox(label="Enter Hidden Text (Optional)")

    with gr.Row():
        depth_slider = gr.Slider(minimum=1, maximum=200, value=30, label="Depth Scale")
        pattern_slider = gr.Slider(minimum=10, maximum=100, value=50, label="Pattern Size")
        resolution_slider = gr.Slider(minimum=100, maximum=1000, value=800, label="Output Width")
        color_selector = gr.Dropdown(
            choices=["grayscale", "viridis", "magma", "plasma", "cividis"],
            value="grayscale",
            label="Color Scheme"
        )

    with gr.Row():
        generate_btn = gr.Button("Generate Stereogram")

    with gr.Row():
        depth_output = gr.Image(label="Depth Map Preview", type="pil")
        pattern_output = gr.Image(label="Pattern Preview", type="pil")
        stereogram_output = gr.Image(label="Generated Stereogram", type="pil")

    generate_btn.click(
        generate_stereogram,
        inputs=[
            depth_input,
            pattern_input,
            logo_input,
            text_input,
            depth_slider,
            pattern_slider,
            color_selector,
            resolution_slider
        ],
        outputs=[depth_output, pattern_output, stereogram_output]
    )

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5dae6e2ae7f174a18b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
